# Model Project

**Table of contents**<a id='toc0_'></a>  
- [1. Objective](#toc1_)
- [2. Import of packages](#toc2_)
- [3. Model description](#toc3_)
- [4. Analytical solution](#toc4_)
- [5. Numerical solution](#toc5_)
- [5. Shock to the model](#toc6_)
- [6. Concluding remarks](#toc6_)

## 1. <a id='toc1_'></a>[Objectives](#toc1_)

## 2. <a id='toc2_'></a>[Import of packages](#toc2_)

## 3. <a id='toc3_'></a>[Reading and cleaning data](#toc3_)

## 4. <a id='toc4_'></a>[Merging datasets](#toc4_)

## 5. <a id='toc5_'></a>[Analysis](#toc5_)

## 6. <a id='toc6_'></a>[Concluding remarks](#toc6_)